<a href="https://colab.research.google.com/github/natarajankrishna/projects/blob/main/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
text=['There is a massive demand for low-weight high strength materials in automotive, space aerospace, and even structural industries in this present engineering world. These industries attract composites only because of their high strength, resistance to wear, and low weight. Among these composites, metal matrix composite finds wide applications due to its elevated properties, excellent resistance property, corrosion resistance, etc. The reinforcements exist in particles, fiber, and whiskers. Among the three, particles play an important role because of their availability and wettability with the metal matrix. Additionally, among the various metal matrices such as aluminum, magnesium, copper, titanium, etc., aluminum plays a vital role among metal matrices because of its cost, availability in abundance, and castability. Stir casting is the most inexpensive and straightforward composite fabrication technique among the prevailing techniques. Even though so many factors contribute to the elevated property of composites, metal matrix, and reinforcement phase, uniform distribution and wettability are essential factors among all the other factors. This review aims to develop a composite with elevated property in a cost-effective manner. Cost includes metal matrix, reinforcement, and processing technique. Various works have been tabulated to achieve the above objective, and analysis was carried out on tensile strength concerning microstructure. This review paper explores the challenges in composite fabrication and finds a solution to overcome them']

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in ['text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [ ]:
word_embeddings = {}
f = 'text'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs


clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

<ipython-input-12-0f05b036608e>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
!pip install --upgrade scipy networkx
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(5):
  print("ARTICLE:")
  print(['text'][i])
  print('\n')
  print("SUMMARY:")
  print(ranked_sentences[i][1])
  print('\n')